# McKinsey Hack-A-Thon

This is code for the one day McKinsey Hack-A-Thon from 3/9/2018 7PM EST to 3/10/2018 7PM EST

# Preliminary Analysis

3 files:

train.csv: 
It contains the set of 13 challenges that were attempted by the same user in a sequence

challenge_data.csv: 
Contains attributes related to each challenge

test.zip: 
Contains the first 10 challenges solved by a new user set (not in train) in the test set. We need to predict the next 3 sequence of challenges for these users.

In [2]:
import pandas as pd
import numpy as np

In [3]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
challenge=pd.read_csv('./challenge_data.csv')

In [4]:
pd.DataFrame({'A':{'u':2}})

,A
u,2


In [5]:
train.head()

,user_sequence,user_id,challenge_sequence,challenge
0,4576_1,4576,1,CI23714
1,4576_2,4576,2,CI23855
2,4576_3,4576,3,CI24917
3,4576_4,4576,4,CI23663
4,4576_5,4576,5,CI23933


In [6]:
from collections import defaultdict
challenge_dict=defaultdict(list)
for i,j in train[['user_id','challenge']].groupby('challenge'):
    challenge_dict[i].extend(j['user_id'].tolist())
for i,j in test[['user_id','challenge']].groupby('challenge'):
    challenge_dict[i].extend(j['user_id'].tolist())

In [7]:
user_dict=defaultdict(list)
for i,j in train[['user_id','challenge']].groupby('user_id'):
    user_dict[i].extend(j['challenge'].tolist())
for i,j in test[['user_id','challenge']].groupby('user_id'):
    user_dict[i].extend(j['challenge'].tolist())

In [8]:
test.head()

,user_sequence,user_id,challenge_sequence,challenge
0,4577_1,4577,1,CI23855
1,4577_2,4577,2,CI23933
2,4577_3,4577,3,CI24917
3,4577_4,4577,4,CI24915
4,4577_5,4577,5,CI23714


In [9]:
challenge.head()

,challenge_ID,programming_language,challenge_series_ID,total_submissions,publish_date,author_ID,author_gender,author_org_ID,category_id
0,CI23478,2,SI2445,37.0,06-05-2006,AI563576,M,AOI100001,NaN
1,CI23479,2,SI2435,48.0,17-10-2002,AI563577,M,AOI100002,32.0
2,CI23480,1,SI2435,15.0,16-10-2002,AI563578,M,AOI100003,NaN
3,CI23481,1,SI2710,236.0,19-09-2003,AI563579,M,AOI100004,70.0
4,CI23482,2,SI2440,137.0,21-03-2002,AI563580,M,AOI100005,NaN


3 main approaches I see:
1. forget time series component and use a matrix factorization technique to recommend such as Alternating Least Squares
2. forget time series component and try to predict next n challenges from previous n challenges
3. Time Series LSTM model to predict next challenge given previous challenges

# Alternating Least Squares(Final Model)

Creating a function to combine knowledge about challenges into Alternating Least Squares:
More weight if similar programming_languages, challenge_series_ID, author_ID and category_id
same programming_language =+2
same challenge_series_ID =+5
same author_ID =+5
same category_id =+5

In [10]:
ch_records=challenge.to_dict(orient='records')

In [11]:
#setting up user_challenge_matrix
challenges_list = challenge['challenge_ID'].tolist()
train_users= set(train['user_id'].tolist())
test_users= set(test['user_id'].tolist())
all_users = list(train_users.union(test_users))

In [12]:
len(all_users)

109264

In [13]:
len(challenges_list)

5606

In [180]:
var_list=[(8,'author_ID'),(8,'challenge_series_ID'),(5,'programming_language'),(.5,'author_gender'),(10,'category_id'),(2,'author_org_ID')] #total_submissions is different


In [181]:
from tqdm import tqdm
from scipy.sparse import coo_matrix
row=[]
col=[]
data=[]
mass_dict=defaultdict(dict)
for i,ch in tqdm(enumerate(challenges_list)):
    c_ch=set(challenge_dict[ch])
    for j,use in enumerate(all_users):
        if use in c_ch:
            row.append(j)
            col.append(i)
            data.append(1)
            if j not in mass_dict.keys():
                mass_dict[j]={}
            for wght,var in var_list:
                if var not in mass_dict[j].keys():
                    mass_dict[j][var]=defaultdict(int)
                mass_dict[j][var][str(ch_records[i][var])]+=1
            if 'total_submissions' in mass_dict[j].keys():
                mass_dict[j]['total_submissions'].append(str(ch_records[i]['total_submissions']))
            else:
                mass_dict[j]['total_submissions']=[str(ch_records[i]['total_submissions'])]
            if 'publish_date' in mass_dict[j].keys():
                mass_dict[j]['publish_date'].append(str(ch_records[i]['publish_date']))
            else:
                mass_dict[j]['publish_date']=[str(ch_records[i]['publish_date'])]

5606it [02:07, 43.97it/s]


In [182]:
from copy import deepcopy
#raise ValueError()
d2=deepcopy(data)

In [282]:
data=deepcopy(d2)

In [283]:
max(data)

1

In [284]:
denom=1

In [292]:
var_list=[(8,'author_ID'),(8,'challenge_series_ID'),(8,'programming_language'),(.5,'author_gender'),(10,'category_id'),(.05,'author_org_ID')] #total_submissions is different
for spc in tqdm(range(len(data))):
    data[spc]/=denom
    user_info=mass_dict[row[spc]]
    current_challenge_info=ch_records[col[spc]]
    for wght,var in var_list:
        if var=='category_id' and not current_challenge_info[var]<=10 and not current_challenge_info[var]>10:
            continue
        data[spc]+=user_info[var][current_challenge_info[var]]*wght/denom
    if not np.isnan(current_challenge_info['total_submissions']):
        data[spc]+=np.log10(current_challenge_info['total_submissions'])*20/denom

100%|██████████████| 1301236/1301236 [00:10<00:00, 125985.14it/s]


In [293]:
from scipy.sparse import coo_matrix
sparse_matrix=coo_matrix((data, (row, col)), shape=(len(all_users), len(challenges_list)))

In [294]:
import implicit
model = implicit.als.AlternatingLeastSquares(factors=250)
model.fit(sparse_matrix.T)

In [295]:
user_items = sparse_matrix.tocsr()

In [296]:
#Test model Recommendation
model.recommend(0,user_items)

[(1658, 0.26384756),
 (1390, 0.26315558),
 (435, 0.25172612),
 (1398, 0.24212229),
 (3414, 0.24014762),
 (134, 0.23387334),
 (2689, 0.23345861),
 (1811, 0.21335006),
 (1646, 0.21297152),
 (1044, 0.20808257)]

In [297]:
submission_1=[]
for i,user in tqdm(enumerate(all_users)):
    if user not in test_users:
        continue
    top3=model.recommend(i,user_items)[:3]
    for j,each in enumerate(top3):
        submission_1.append([str(user)+'_'+str(11+j),challenges_list[each[0]]])

109264it [00:18, 6011.32it/s]


In [298]:
submission_1=pd.DataFrame(submission_1,columns=['user_sequence','challenge'])
submission_1.to_csv('submission_1.csv',index=False)

17.7% accuracy predicting next three challenges a user would do (scored based on Mean Average Precision (MAP) )

# Stuff that didn't work out below:

# LightFM Model

In [155]:
from tqdm import tqdm
from scipy.sparse import coo_matrix
row=[]
col=[]
data=[]
mass_dict=defaultdict(dict)
for i,ch in tqdm(enumerate(challenges_list)):
    c_ch=set(challenge_dict[ch])
    for j,use in enumerate(all_users):
        if use in c_ch:
            row.append(j)
            col.append(i)
            data.append(1)
            if j not in mass_dict.keys():
                mass_dict[j]={}
            for wght,var in var_list:
                if var not in mass_dict[j].keys():
                    mass_dict[j][var]=defaultdict(int)
                mass_dict[j][var][str(ch_records[i][var])]+=1
            if 'total_submissions' in mass_dict[j].keys():
                mass_dict[j]['total_submissions'].append(str(ch_records[i]['total_submissions']))
            else:
                mass_dict[j]['total_submissions']=[str(ch_records[i]['total_submissions'])]
            if 'publish_date' in mass_dict[j].keys():
                mass_dict[j]['publish_date'].append(str(ch_records[i]['publish_date']))
            else:
                mass_dict[j]['publish_date']=[str(ch_records[i]['publish_date'])]

5606it [02:05, 53.54it/s]


In [156]:
denom=20

In [157]:
var_list=[(8,'author_ID'),(8,'challenge_series_ID'),(5,'programming_language'),(.5,'author_gender'),(10,'category_id'),(2,'author_org_ID')] #total_submissions is different
for spc in tqdm(range(len(data))):
    data[spc]/=denom
    user_info=mass_dict[row[spc]]
    current_challenge_info=ch_records[col[spc]]
    for wght,var in var_list:
        if var=='category_id' and not current_challenge_info[var]<=10 and not current_challenge_info[var]>10:
            continue
        data[spc]+=user_info[var][current_challenge_info[var]]*wght/denom
    if not np.isnan(current_challenge_info['total_submissions']):
        data[spc]+=np.log10(current_challenge_info['total_submissions'])*10/denom

100%|██████████████| 1301236/1301236 [00:09<00:00, 132300.97it/s]


In [158]:
from scipy.sparse import coo_matrix
sparse_matrix=coo_matrix((data, (row, col)), shape=(len(all_users), len(challenges_list)))

In [121]:
challenge.columns

Index(['challenge_ID', 'programming_language', 'challenge_series_ID',
       'total_submissions', 'publish_date', 'author_ID', 'author_gender',
       'author_org_ID', 'category_id'],
      dtype='object')

In [161]:
#Generate item features
cols=[]
col_set=set()
for i in ch_records:
    v = [i[vb] for vb in ['challenge_series_ID','total_submissions','author_org_ID','author_gender','category_id']]
    for val in v:
        if type(val)==float:
            if str(val)=='nan':
                v=0
            else:
                v=str(int(np.log10(val)))
        else:
            v=str(val)
        if v in col_set:
            continue
        else:
            cols.append(v)
            col_set.add(v)

In [162]:
len(cols)

2160

In [163]:
colmap={j:i for i,j in enumerate(cols)}

In [167]:
item_features=[]
item_rows=[]
item_cols=[]
item_data=[]
for ind,i in enumerate(ch_records):
    v = [i[vb] for vb in ['challenge_series_ID','total_submissions','author_org_ID','author_gender','category_id']]
    for val in v:
        if type(val)==float:
            if str(val)=='nan':
                v=0
            else:
                v=str(int(np.log10(val)))
        else:
            v=str(val)
        item_rows.append(ind)
        item_cols.append(colmap[v])
        item_data.append(1)

In [168]:
item_features=coo_matrix((item_data, (item_rows, item_cols)), shape=(len(ch_records), len(colmap)))

In [169]:
item_features.shape

(5606, 2160)

In [170]:
sparse_matrix.shape

(109264, 5606)

In [171]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k

In [172]:
# Instantiate and train the model
model = LightFM(loss='warp')

In [173]:
model.fit(sparse_matrix, item_features=item_features,epochs=200, num_threads=20)

In [178]:
train_precision = precision_at_k(model, sparse_matrix,item_features=item_features, k=3,num_threads=10).mean()

In [179]:
train_precision

0.23810831

In [174]:
submission_2=[]
for i,user in tqdm(enumerate(all_users)):
    if user not in test_users:
        continue
    scores=model.predict(user_ids=np.array([i]*len(ch_records)),
              item_ids=np.array(range(len(ch_records))),
              item_features=item_features,num_threads=5)
    sc_list=list(zip(range(len(scores)),scores))
    top40=sorted(sc_list, key= lambda x: x[1],reverse=True)[:40]
    zx=0
    for j,each in enumerate(top40):
        if challenges_list[each[0]] in user_dict[int(user)]:
            continue
        else:
            zx+=1
            submission_2.append([str(user)+'_'+str(10+zx),challenges_list[each[0]]])
        if zx==3:
            break


109264it [03:33, 510.96it/s]


In [175]:
submission_2=pd.DataFrame(submission_2,columns=['user_sequence','challenge'])

In [176]:
submission_2.head()

,user_sequence,challenge
0,4577_11,CI24944
1,4577_12,CI24957
2,4577_13,CI25733
3,4578_11,CI24944
4,4578_12,CI24957


In [177]:
submission_2.to_csv('submission_2.csv',index=False)

In [597]:
from lightfm.datasets import fetch_movielens
movielens = fetch_movielens()
for key, value in movielens.items():
    print(key, type(value), value.shape)

train <class 'scipy.sparse.coo.coo_matrix'> (943, 1682)
test <class 'scipy.sparse.coo.coo_matrix'> (943, 1682)
item_features <class 'scipy.sparse.csr.csr_matrix'> (1682, 1682)
item_feature_labels <class 'numpy.ndarray'> (1682,)
item_labels <class 'numpy.ndarray'> (1682,)


# Simple LSTM Model

To Do:

Check to see if one-hot is best option or if beginnings of some of the IDs mean something

Create LSTM

In [ ]:
data=[]
for i,j in train[['user_id','challenge']].groupby('user_id'):
    for chal in j['challenge'].tolist():
        one_hot=np.zeros(len(challenge_list))
        one_hot[challenge_list.index(chal)]=1
        challenge[challenge_ID==chal]
    user_dict[i].extend(j['challenge'].tolist())